# Les fonctions

Il est possible de définir une fonction de trois façons différentes :

## Définition en ligne

In [1]:
f(x) = 3x + 1

f (generic function with 1 method)

In [2]:
f(2)

7

In [3]:
g(x,y)=[x*y+1, x-y]

g (generic function with 1 method)

In [4]:
g(1,2)

2-element Vector{Int64}:
  3
 -1

Il est possible d'utiliser **begin**-**end** ou **( ; )** pour délimiter la fonction la dernière valeur calculée est retournée

In [5]:
h(x) = begin
    y = 2
    x + y
end

h (generic function with 1 method)

In [6]:
h(3)

5

In [7]:
h(x) = (y=2; x+y) # equivalent à la première écriture

h (generic function with 1 method)

In [8]:
h(3)

5

## Structurée

*Julia* possède une structure plus classique à l'aide de **function**-**end** comme précédemment la dernière valeur calculée est par défaut la variable de retour autrement l'utilisation de **return** spécifie la ou les variables de sortie. 

In [9]:
function h(x,y)
    z = x+y
    return z^2/(abs(x)+1)
end

h (generic function with 2 methods)

In [10]:
h(1,2)

4.5

L'usage de **return** pour fixer la sortie 

In [11]:
function choix(x)
    if x>0
        return "Positif"
    else
        return "Négatif"
    end
end

choix (generic function with 1 method)

In [12]:
txt=choix(3)

"Positif"

In [13]:
choix(x) = x > 0 ? "Positif" : "Négatif"

choix (generic function with 1 method)

## Anonyme

Le concept peut paraître abstrait mais on peut définir une fonction sans la nommer puis l'affecter à une variable...

In [14]:
x -> x^2

#1 (generic function with 1 method)

In [17]:
λ = x -> sqrt(x)

#5 (generic function with 1 method)

In [18]:
λ(1)

1.0

In [20]:
typeof(λ)

var"#5#6"

## Arguments de sortie

Pour avoir plusieurs arguments de sortie il faut utiliser un "tuple" autrement dit une collection d'éléments

In [21]:
function multi_output(x,y)
    return x+y, x-y
end

multi_output (generic function with 1 method)

In [22]:
a = multi_output(1,2) # un seul argument de sortie qui contient un tuple

(3, -1)

In [23]:
typeof(a)

Tuple{Int64, Int64}

In [24]:
a[1]

3

In [25]:
a[2]

-1

In [26]:
a, b = multi_output(1,2) # assignation aux 2 arguments de sortie

(3, -1)

In [27]:
a

3

In [28]:
b

-1

## Portée des variables 

Quelle est la portée des variables, autrement dit une variable définie peut elle être accessible, modifiable dans une fonction sans la passer en paramètre ?

In [29]:
a = 1
f(x) = a + x
f(1)

2

Attention, lorsque l'on utilise des variables globales à l'intérieur d'une fonction. La compilation est plus difficile car
la fonction doit prendre en compte les modifications éventuelles de la variable `a`

In [30]:
function ff1(x)
    x + a
end

@code_llvm ff1(1)

;  @ In[30]:1 within `ff1`
define nonnull {}* @julia_ff1_1363(i64 signext %0) #0 {
top:
  %1 = alloca [2 x {}*], align 8
  %gcframe2 = alloca [4 x {}*], align 16
  %gcframe2.sub = getelementptr inbounds [4 x {}*], [4 x {}*]* %gcframe2, i64 0, i64 0
  %.sub = getelementptr inbounds [2 x {}*], [2 x {}*]* %1, i64 0, i64 0
  %2 = bitcast [4 x {}*]* %gcframe2 to i8*
  call void @llvm.memset.p0i8.i64(i8* align 16 %2, i8 0, i64 32, i1 true)
  %thread_ptr = call i8* asm "movq %fs:0, $0", "=r"() #6
  %tls_ppgcstack = getelementptr i8, i8* %thread_ptr, i64 -8
  %3 = bitcast i8* %tls_ppgcstack to {}****
  %tls_pgcstack = load {}***, {}**** %3, align 8
;  @ In[30]:2 within `ff1`
  %4 = bitcast [4 x {}*]* %gcframe2 to i64*
  store i64 8, i64* %4, align 16
  %5 = getelementptr inbounds [4 x {}*], [4 x {}*]* %gcframe2, i64 0, i64 1
  %6 = bitcast {}** %5 to {}***
  %7 = load {}**, {}*** %tls_pgcstack, align 8
  store {}** %7, {}*** %6, align 8
  %8 = bitcast {}*** %tls_pgcstack to {}***
  store {}** 

Lorsqu'une variable est globale il est conseillé de lui donner un caractère constant. Le compilateur vous remerciera dans ce cas.

In [31]:
const α = 1
function ff2(x)
    x + α
end

@code_llvm ff2(1)

;  @ In[31]:2 within `ff2`
define i64 @julia_ff2_1388(i64 signext %0) #0 {
top:
;  @ In[31]:3 within `ff2`
; ┌ @ int.jl:87 within `+`
   %1 = add i64 %0, 1
; └
  ret i64 %1
}


In [32]:
@code_lowered ff2(1)

CodeInfo(
1 ─ %1 = x + Main.α
└──      return %1
)

Donc par défaut une variable définie est connue et utilisable par toute fonction appelée (de même à l'intérieur d'une fonction).

Si on redéfinit localement dans la fonction la variable alors "elle écrase localement" la dite variable et en sortie de fonction rien n'est modifié.

Attention à l'utilisation dans la fonction d'une variable extérieure puis d'affecter une valeur à cette variable...



## Le mapping

Souvent on écrit une fonction sous une forme scalaire sans chercher à utiliser les opérations vectorielles pour appliquer cette fonction à un tableau

In [33]:
f(x) = x^2 + 1
f(1:5)  # il aurait fallu définir f(x)=x.^2.+1

LoadError: MethodError: no method matching ^(::UnitRange{Int64}, ::Int64)

[0mClosest candidates are:
[0m  ^([91m::Float16[39m, ::Integer)
[0m[90m   @[39m [90mBase[39m [90m[4mmath.jl:1283[24m[39m
[0m  ^([91m::Regex[39m, ::Integer)
[0m[90m   @[39m [90mBase[39m [90m[4mregex.jl:863[24m[39m
[0m  ^([91m::Float32[39m, ::Integer)
[0m[90m   @[39m [90mBase[39m [90m[4mmath.jl:1277[24m[39m
[0m  ...


La fonction <code>map</code> permet de palier à ce manquement

In [34]:
map(f,1:5)

5-element Vector{Int64}:
  2
  5
 10
 17
 26

In [35]:
f.(1:5)

5-element Vector{Int64}:
  2
  5
 10
 17
 26

In [36]:
v = [1 2; 3 4]
map(f,v)

2×2 Matrix{Int64}:
  2   5
 10  17

In [37]:
reduce(+, v )

10

In [38]:
sum(v)

10

In [39]:
f.([1 2; 3 4])

2×2 Matrix{Int64}:
  2   5
 10  17

In [40]:
g(x,y)=x+y
map(g,0:3,1:4)

4-element Vector{Int64}:
 1
 3
 5
 7

In [41]:
map(g,[1 2;3 4],[2 3;4 5])

2×2 Matrix{Int64}:
 3  5
 7  9

In [42]:
g(f,x)=f(x)+1

g (generic function with 1 method)

In [43]:
g(sin,1)

1.8414709848078965

Il existe également un opérateur pour "composer" les fonctions `\circ`

In [44]:
f1(x) = 2x + 1
f2(x) = 1 - 4x
(f1 ∘ f2)(3)

-21

In [45]:
f1(f2(3))

-21